# Modifying cytosim objects
Requires Python>=3.7  
Before running, copy or move the cytosim module (cytosim.---.so) to the current folder.

Here we showcase by changing a fiber.

In [1]:
import cytoplay
import numpy as np


## Fiber simulation
Here we simulate and a fiber. Each point correspond to the end of a segment.

In [2]:
sim = cytoplay.start("hero.cym")



In [3]:
dt = sim.time_step()
dt

0.004

In [4]:
frame = sim.frame()
hero = frame["good"][0]
bads = frame["bad"]
pts =  np.array(hero.data(), copy=False)
badies = [np.array(bad.data(), copy=False) for bad in bads]
speedos = [dt*20.0*(np.random.rand(1,2)-0.5) for bad in bads]

In [5]:
speed = np.zeros((1,2))
acc = np.zeros((1,2))

In [6]:
maxspeed = 20.0
decel = 10*dt

In [7]:
distSqr = np.square(bads[0].radius() + hero.radius())

In [8]:
def runtime_all(s, speed, pts):
    for i,pt in enumerate(badies): 
        pt += speedos[i]
        if np.sum(np.square(pts-pt)) < distSqr:
            sim.change("good","display = (color = red)")

    
    speed -= speed*decel
    speed += acc*dt
    ns = np.sqrt(np.sum(np.square(speed)))
    if ns>maxspeed:
        speed *= maxspeed/ns
    pts += speed*dt
    acc[:,:] = 0
    

In [9]:
runtime = lambda s:runtime_all(s, speed, pts)

In [10]:
def key_cb(key, i, j):
    changed = 1
    mult = 5000
    if key==113:
        acc[:,0]-=mult
    elif key==100:
        acc[:,0]+=mult
    elif key==122:
        acc[:,1]+=mult
    elif key==115:
        acc[:,1]-=mult
    else:
        return key
    return 0
        
        

In [11]:
cytoplay.setNormalKey(key_cb)
cytoplay.setRuntimeCheck(runtime)

In [ ]:
try:
    cytoplay.play()
except:
    print("Simulation crashed or ended.")